<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/jacob-dev/MitelmanDB/Mitelman_DB_Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing Complex Queries in the Mitelman Database Using Joins

Check out other notebooks at our [Community Notebooks Repository](https://github.com/isb-cgc/Community-Notebooks)!

- Title: Performing Complex Queries in the Mitelman Database Using Joins
- Author: Jacob Wilson
- Created: 2025-01-01
- URL: https://github.com/isb-cgc/Community-Notebooks/blob/jacob-dev/MitelmanDB/Mitelman_DB_Joins.ipynb
- Purpose: Extend our capabilities in obtaining data in the Mitelman Datbase BigQuery tables by performing table joins.
<br/>

In this notebook, we will use SQL joins to combine BigQuery tables in the Mitelman Database. This will allow for us to perform more complex operations for obtaining and aggregating our data.

## Background on SQL Joins
In a previous notebook we demonstrated how to find basic information in the Mitelman Database BigQuery tables by using SQL. Those examples were limited to finding data in a single table. We will now demonstrate the SQL join clause, which can be used to join data across multiple tables based on a related column. There are various types of joins and their use also depends on your system, but they mostly follow the same principles.

The general syntax of a join is:

    SELECT a.column1, b.column2
    FROM table a
    JOIN table b
    ON a.id = b.id

Multilpe types of joins are available which return differing combined tables:

- INNER JOIN: all rows that meet the conditions of the join
- RIGHT OUTER JOIN: all rows from the right side of the join with matched rows from the left
- LEFT OUTER JOIN: all rows from the left side of the join with matched rows from the right
- FULL OUTER JOIN: all rows from both tables, rows are combined when the join criteria matches
- CROSS JOIN: all rows from both tables as a Cartesian product (all possible row combinations)

See more about using joins:

https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax
https://cloud.google.com/blog/topics/developers-practitioners/bigquery-explained-working-joins-nested-repeated-data


## Initialize Notebook Environment

Before beginning, we first need to load dependencies and authenticate to BigQuery.

## Install Dependencies

In [1]:
# GCP libraries
from google.cloud import bigquery
from google.colab import auth

## Authenticate

In order to utilize BigQuery, we must obtain authorization to BigQuery and Google Cloud.

In [2]:
# if you're using Google Colab, authenticate to gcloud with the following
auth.authenticate_user()

# alternatively, use the gcloud SDK
#!gcloud auth application-default login

## Google project ID

Set your own Google project ID for use with this notebook.

In [3]:
# set the google project that will be billed for this notebook's computations
google_project = 'your_project_id'  ## change this

## BigQuery Client

In [4]:
# Initialize a client to access the data within BigQuery
if google_project == 'your_project_id':
    print('Please update the project ID with your Google Cloud Project')
else:
    client = bigquery.Client(google_project)

# set the Mitelman Database project
bq_project = 'mitelman-db'
bq_dataset = 'prod'

## Simple Join Examples

Using joins requires having entries in the individual tables that can be matched by a unique identifier. In the Mitelman database, that identifier is often the RefNo (Reference Number), CaseNo (Case Number), and InvNo (Investigation Number). The examples below will use the RefNo to return matching data from two tables.

In [5]:
# Performing a left join will include all rows from the left-side table
# (Reference table in this example). The result of the query below will be a
# table containing a matching PubMed ID or Name for every title. If a PubMed
# ID or name is missing, NULL will be entered into the field
left_join_example = f'''SELECT r.titleShort, r.Pubmed, a.Name
FROM `{bq_project}.{bq_dataset}.Reference` r
LEFT JOIN `{bq_project}.{bq_dataset}.AuthorReference` a
ON r.RefNo = a.RefNo'''

# If we use this same example as a right join, all rows from the right-side
# table (AuthorReference in this example) will be included. In this case, every
# name will be included and missing titles or PubMed ID's will be indicated
# with NULL.
right_join_example = f'''SELECT a. Name, r.titleShort, r.Pubmed
FROM `{bq_project}.{bq_dataset}.Reference` r
RIGHT JOIN `{bq_project}.{bq_dataset}.AuthorReference` a
ON r.RefNo = a.RefNo'''

In [6]:
df_left_join = client.query(left_join_example).result().to_dataframe()
df_right_join = client.query(right_join_example).result().to_dataframe()

In [12]:
print(df_left_join.head())
print(df_right_join.head())

           name                                         titleShort   Pubmed
0   Hellstrom K  An extra C chromosome and various metabolic ab...  5134476
1  Hagenfeldt L  An extra C chromosome and various metabolic ab...  5134476
2     Larsson A  An extra C chromosome and various metabolic ab...  5134476
3    Lindsten J  An extra C chromosome and various metabolic ab...  5134476
4    Sundelin P  An extra C chromosome and various metabolic ab...  5134476
           name                                         titleShort   Pubmed
0      Hayata I  Another translocation related to the Ph1 chrom...    48930
1      Muldal S  A new translocation associated with the Ph1 ch...  1057443
2      Hayata I  A new translocation related to the Philadelphi...  4128072
3    Mitelman F  Heterogeneity of Ph1 in chronic myeloid leukae...  4135826
4  Shah-Reddy I  Sézary syndrome with a 14:14(q12:q31) transloc...  7053821


## More Complex Joins

In [8]:
# Fields such as morphology, topography, and country are identified in the
# Mitelman database using unique codes. To obtain the full names of these
# features, you can join to the Koder table as demonstrated in these
# examples. The following examples use JOIN, which is the same as INNER JOIN.
morph = f'''SELECT m.Morph AS Morph_Code, k.Benamning AS Morph_Name
FROM `{bq_project}.{bq_dataset}.MolBiolClinAssoc` m
JOIN `{bq_project}.{bq_dataset}.Koder` k
  ON k.Kod = m.Morph AND k.kodTyp = "MORPH"
GROUP BY m.Morph, k.Benamning
ORDER BY k.Benamning ASC'''

topo = f'''SELECT m.Topo AS Topo_Code, k.Benamning AS Topo_Name
FROM `{bq_project}.{bq_dataset}.MolBiolClinAssoc` m
JOIN `{bq_project}.{bq_dataset}.Koder` k
  ON k.Kod = m.Topo AND k.kodTyp = "TOP"
GROUP BY m.Topo, k.Benamning
ORDER BY k.Benamning ASC'''

country = f'''SELECT c.Country AS Country_Code, k.Benamning AS Country_Name
FROM `{bq_project}.{bq_dataset}.Cytogen` c
JOIN `{bq_project}.{bq_dataset}.Koder` k
  ON k.Kod = c.Country AND k.kodTyp = "GEO"
GROUP BY c.Country, k.Benamning
ORDER BY k.Benamning ASC'''

# By joining tables and using other SQL features, you can perform more
# complex queries such as finding the ten most common gene fusions for
# prostate adenocarcinoma as demonstrated here.
fusions_prostate = f'''
SELECT g.Gene, count(g.Gene) AS Count, m.Morph, k.Benamning, m.Topo
FROM `{bq_project}.{bq_dataset}.MolClinGene` g
JOIN `{bq_project}.{bq_dataset}.MolBiolClinAssoc` m
  ON m.RefNo = g.RefNo AND m.InvNo = g.InvNo
JOIN `{bq_project}.{bq_dataset}.Koder` k
  ON k.Kod = m.Topo AND k.kodTyp = "TOP"
-- we are only considering gene fusions for Prostate adenocarcinoma
WHERE g.Gene LIKE "%::%" AND m.Morph LIKE "3111" AND m.Topo LIKE "0602"
GROUP BY g.Gene, m.Morph, k.Benamning, m.Topo
ORDER BY Count DESC
LIMIT 10
'''

In [9]:
df_morph = client.query(morph).result().to_dataframe()
df_topo = client.query(topo).result().to_dataframe()
df_country = client.query(country).result().to_dataframe()
df_fusions_prostate = client.query(fusions_prostate).result().to_dataframe()

In [11]:
print(df_morph.head())
print(df_topo.head())
print(df_country.head())
print(df_fusions_prostate.head())

  Morph_Code                                         Morph_Name
0       3117                              Acinic cell carcinoma
1       1115                          Acute basophilic leukemia
2       1117                        Acute eosinophilic leukemia
3       1112                Acute erythroleukemia (FAB type M6)
4       1602  Acute lymphoblastic leukemia/lymphoblastic lym...
  Topo_Code   Topo_Name
0      0703     Adrenal
1      0230        Anus
2      0305     Bladder
3      0801       Brain
4      0806  Brain stem
  Country_Code   Country_Name
0         4003    Afghanistan
1         1000  Africa unspec
2         1010        Algeria
3         2010      Argentina
4         4005        Armenia
            Gene  Count Morph Benamning  Topo
0   TMPRSS2::ERG     52  3111  Prostate  0602
1  TMPRSS2::ETV1      5  3111  Prostate  0602
2   SLC45A3::ERG      5  3111  Prostate  0602
3  TMPRSS2::ETV4      5  3111  Prostate  0602
4     NDRG1::ERG      4  3111  Prostate  0602


## Conclusion

Modern databases are often complex, so joins are likely to be one of the most common SQL functions that you will be using. As you create more complex SQL queries, you may find it useful to store the results as a table. See the BigQuery Views notebook to learn more about that.